## [C++] xtensor

We can use `xtensor` C++ library natively in the Jupyter notebook with the xeus-cling kernel

In [ ]:
%%file demo.cpp

#include <numeric>               // Standard library import for std::accumulate
#include <xtensor/xmath.hpp>     // xtensor import for the C++ universal functions

template <class E>
double sum_of_sines(E m)
{
    auto sines = xt::sin(m);
    // return xt::sum(sines)();
    return std::accumulate(sines.begin(), sines.end(), 0.0);
}

In [ ]:
>xcpp14

#include <iostream>
#include "demo.cpp"

std::cout << "HELLO C++ 14! " << sum_of_sines(xt::arange<double>(10000.0));

## [Python] xtensor-python

We can expose the `sum_of_sines` to Python with `pybind11` and `xtensor-python`.

```cpp
#ifdef XPYTHON
#include "pybind11/pybind11.h"            // Pybind11 import to define Python bindings
#define FORCE_IMPORT_ARRAY
#include "xtensor-python/pyarray.hpp"     // Numpy bindings

PYBIND11_MODULE(xpython, m)
{
    xt::import_numpy();
    m.doc() = "Test module for xtensor python bindings";

    m.def("sum_of_sines", sum_of_sines<xt::pyarray<double, xt::layout_type::row_major>>,
                          "Sum the sines of the input values");
}
#endif
```

In [ ]:
%%bash
rm -f xpython.cpp

cp demo.cpp xpython.cpp

cat << EOF >> xpython.cpp
#include "pybind11/pybind11.h"            // Pybind11 import to define Python bindings
#define FORCE_IMPORT_ARRAY
#include "xtensor-python/pyarray.hpp"     // Numpy bindings

PYBIND11_MODULE(xpython, m)
{
    xt::import_numpy();
    m.doc() = "Test module for xtensor python bindings";

    m.def("sum_of_sines", sum_of_sines<xt::pyarray<double, xt::layout_type::row_major>>,
                          "Sum the sines of the input values");
}
EOF

In [ ]:
%%bash

export PYTHON_INCLUDES=`python3-config --includes`
export NUMPY_INCLUDES=`python -c "import numpy; print(numpy.get_include())"`

$CXX -w -shared -fPIC \
     -I${CONDA_PREFIX}/include \
     -L${CONDA_PREFIX}/lib/ \
     -I${NUMPY_INCLUDES} \
     ${PYTHON_INCLUDES} \
     xpython.cpp -o xpython.so

In [ ]:
>python3
import numpy as np
from xpython import sum_of_sines

x = np.arange(10000.0)
print("NumPy  : ", np.sum(np.sin(x)))
print("xtensor: ", sum_of_sines(x))

## [R] xtensor-r

We can expose the `sum_of_sines` to R with `Rcpp` and `xtensor-r`.

```c++
#include <xtensor-r/rarray.hpp>       // R bindings
#include <Rcpp.h>

using namespace Rcpp;

// [[Rcpp::plugins(cpp14)]]
// [[Rcpp::export]]
double sum_of_sines(const xt::rarray<double>& e)
{
    return sum_of_sines_impl(e);
}
#endif
```

In [ ]:
%%bash
rm -f xr.cpp
cp demo.cpp xr.cpp

cat << EOF >> xr.cpp
#include <xtensor-r/rarray.hpp>       // R bindings
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::plugins(cpp14)]]
// [[Rcpp::export]]
double rsum_of_sines(const xt::rarray<double>& e) {
    return sum_of_sines(e);
}
EOF

In [ ]:
>ir
conda_prefix <- Sys.getenv("CONDA_PREFIX")
Sys.setenv(PKG_CXXFLAGS=paste("-I", conda_prefix, "/include/", sep=""))
Rcpp::sourceCpp("xr.cpp");

arr <- seq(0., 10000.0 - 1.0, 1.0)
rsum_of_sines(arr)

## [Julia]  xtensor-julia

We can expose the `sum_of_sines` to Julia with `CxxWrap` and `xtensor-julia`.

```c++
#include "jlcxx/jlcxx.hpp"               // libcxxwrap import to define Julia bindings
#include "xtensor-julia/jlarray.hpp"     // Import the jltensor container definition

JLCXX_MODULE define_julia_module(jlcxx::Module& mod)
{
    mod.method("sum_of_sines", sum_of_sines);
}
```

Then in Julia

```julia
module xjulia
  using CxxWrap
  @wrapmodule(joinpath(".", "xjulia"))
  function __init__()
    @initcxx
  end
end
```

In [ ]:
%%bash
rm -f xjulia.cpp
cp demo.cpp xjulia.cpp

cat << EOF >> xjulia.cpp
#include "jlcxx/jlcxx.hpp"
#include "xtensor-julia/jlarray.hpp"

void foo() { static std::allocator<double> al; }

JLCXX_MODULE define_julia_module(jlcxx::Module& mod)
{
    mod.method("sum_of_sines", sum_of_sines<xt::jlarray<double>>);
}

EOF

In [ ]:
%%bash

$CXX -w -shared -fPIC \
     -I${CONDA_PREFIX}/include \
     -L${CONDA_PREFIX}/lib/ \
     -I${CONDA_PREFIX}/include/julia \
     -DJULIA_ENABLE_THREADING -ldl -lpthread \
     -lcxxwrap_julia \
     xjulia.cpp -o xjulia.so

In [ ]:
>julia-1.0
# This cell may take a long time to execute because of the IJulia precompilation

module xjulia
  using CxxWrap
  @wrapmodule(joinpath(".", "xjulia"))
  function __init__()
    @initcxx
  end
end

In [ ]:
>julia-1.0

xjulia.sum_of_sines(collect(0.0:1.0:9999))